# Neural Net vs. Goldman
Can an LTSM Neural Net trained on fundamentals extracted from edgar XBRL from the S&P500 pick the same long and short list as Goldman's hedgefund meta list (appologies to my high school english teacher for the run on sentence)

## Data Sources
Download price and fundamental data for the S&P 500 using [pystock-crawler](https://github.com/eliangcs/pystock-crawler):  

`pystock-crawler reports ../tickers.csv -o ../reports.csv ../reports.log`  
`pystock-crawler prices ../tickers.csv -o ../prices.csv -l ../prices.log`

REMIND: Use pystock-crawler symbols to get symbols for training input...

X = 4 quarters of fundamental data and whether stock was up or down from the prior quarter y = whether the stock was up or down x days after the period that X was comprised of

In [117]:
import pandas
prices = pandas.read_csv('prices.csv', parse_dates=['date'], index_col=1)
reports = pandas.read_csv('reports.csv', parse_dates=['date'], index_col=1)
symbols = pandas.read_csv('symbols.csv')

In [128]:
reports[reports.amend == True]

,symbol,amend,period_focus,fiscal_year,doc_type,revenues,op_income,net_income,eps_basic,eps_diluted,dividend,assets,cur_assets,cur_liab,cash,equity,cash_flow_op,cash_flow_inv,cash_flow_fin
date,,,,,,,,,,,,,,,,,,,
2014-03-31,GAS,True,Q1,2014,10-Q,2.462000e+09,592000000,284000000,2.39,2.38,0.4900,1.526400e+10,3589000000,3754000000,267000000,3.843000e+09,853000000,-164000000,-501000000
2014-09-30,GAS,True,Q3,2014,10-Q,5.890000e+08,78000000,-8000000,-0.06,-0.06,0.4900,1.395300e+10,2093000000,2462000000,32000000,3.802000e+09,874000000,-284000000,-663000000
2014-06-30,GAS,True,Q2,2014,10-Q,8.890000e+08,139000000,58000000,0.49,0.49,0.4900,1.418100e+10,2384000000,2595000000,122000000,3.860000e+09,1175000000,-300000000,-843000000
2013-12-31,GAS,True,FY,2013,10-K,4.209000e+09,639000000,295000000,2.50,2.49,1.8800,1.455000e+10,2895000000,3118000000,81000000,3.613000e+09,971000000,-876000000,-121000000
2009-07-31,A,True,Q3,2009,10-Q,1.057000e+09,-1000000,-19000000,-0.06,-0.06,0.0000,6.573000e+09,2897000000,991000000,1479000000,2.489000e+09,195000000,34000000,-104000000
2012-12-31,AXP,True,FY,2012,10-K,2.959200e+10,NaN,4482000000,3.91,3.89,0.8000,1.531400e+11,NaN,NaN,22250000000,1.888600e+10,7082000000,-6545000000,-3268000000
2013-12-31,AXP,True,FY,2013,10-K,3.086400e+10,NaN,5359000000,4.91,4.88,0.8900,1.533750e+11,NaN,NaN,19486000000,1.949600e+10,8547000000,-7269000000,-3891000000
2014-12-31,AXP,True,FY,2014,10-K,3.224800e+10,NaN,5885000000,5.58,5.56,1.0100,1.591030e+11,NaN,NaN,22288000000,2.067300e+10,10990000000,-7967000000,11000000
2009-09-26,AAPL,True,FY,2009,10-K,4.290500e+10,11740000000,8235000000,9.22,9.08,0.0000,4.750100e+10,31555000000,11506000000,5263000000,3.164000e+10,10159000000,-17434000000,663000000


In [127]:
# Create a set of sequences of reports excluding amended
s = 'MMM'
r = reports[reports.symbol == s].sort(ascending=True)
p = prices[prices.symbol == s].sort(ascending=True)
print "Found", len(r), "reports for", s

Found 24 reports for MMM


In [121]:
# Add closing stock price to each report
r['close'] = r.index.map(lambda x: p.ix[p.index[p.index.searchsorted(x)]]['close'])

In [122]:
# Fixup annual 10-k numbers by subtracting the prior 3 quarters
for c in ['revenues', 'op_income', 'net_income',
 'eps_basic', 'eps_diluted', 'dividend',
 'cash_flow_op', 'cash_flow_inv', u'cash_flow_fin']:
    r[c + '_adj'] = r[c] - r[c].shift(1) - r[c].shift(2) - r[c].shift(3)
    r.ix[r.period_focus == 'FY', c] = r[r.period_focus == 'FY'][c + '_adj']

In [123]:
r

,symbol,amend,period_focus,fiscal_year,doc_type,revenues,op_income,net_income,eps_basic,eps_diluted,...,close,revenues_adj,op_income_adj,net_income_adj,eps_basic_adj,eps_diluted_adj,dividend_adj,cash_flow_op_adj,cash_flow_inv_adj,cash_flow_fin_adj
date,,,,,,,,,,,,,,,,,,,,,
2009-06-30,MMM,False,Q2,2009,10-Q,5719000000,1191000000,783000000,1.12,1.12,...,60.10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-09-30,MMM,False,Q3,2009,10-Q,6193000000,1478000000,957000000,1.36,1.35,...,73.80000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-12-31,MMM,False,FY,2009,10-K,NaN,NaN,NaN,NaN,NaN,...,82.67000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-31,MMM,False,Q1,2010,10-Q,6348000000,1445000000,930000000,1.31,1.29,...,83.57000,-28687000000,-6038000000,-4003000000,-5.73,-5.70,-2.535,-9917000000,2196000000,4283000000
2010-06-30,MMM,False,Q2,2010,10-Q,6731000000,1596000000,1121000000,1.57,1.54,...,78.99000,-28933000000,-6141000000,-3959000000,-5.66,-5.62,-2.550,-7700000000,2357000000,3008000000
2010-09-30,MMM,False,Q3,2010,10-Q,6874000000,1576000000,1106000000,1.55,1.53,...,86.71000,-29328000000,-6279000000,-4138000000,-5.89,-5.82,-2.565,-4700000000,3217000000,1825000000
2010-12-31,MMM,False,FY,2010,10-K,6709000000,1301000000,928000000,1.29,1.27,...,86.30000,6709000000,1301000000,928000000,1.29,1.27,0.525,-1671000000,676000000,14000000
2011-03-31,MMM,False,Q1,2011,10-Q,7311000000,1578000000,1081000000,1.52,1.49,...,93.50000,-32956000000,-7512000000,-5231000000,-7.32,-7.21,-2.600,-10204000000,4310000000,3317000000
2011-06-30,MMM,False,Q2,2011,10-Q,7680000000,1655000000,1160000000,1.63,1.60,...,94.85000,-33167000000,-7417000000,-5112000000,-7.16,-7.05,-2.625,-7266000000,3577000000,2510000000


In [115]:
# Delete all non-numeric columns
r = r.ix[:,5:-1]

In [116]:
r

,revenues,op_income,net_income,eps_basic,eps_diluted,dividend,assets,cur_assets,cur_liab,cash,...,cash_flow_inv,cash_flow_fin,revenues_adj,op_income_adj,net_income_adj,eps_basic_adj,eps_diluted_adj,dividend_adj,cash_flow_op_adj,cash_flow_inv_adj
date,,,,,,,,,,,,,,,,,,,,,
2009-06-30,5719000000,1191000000,783000000,1.12,1.12,0.510,26095000000,9898000000,5318000000,2596000000,...,-375000000,-1127000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-09-30,6193000000,1478000000,957000000,1.36,1.35,0.510,27633000000,11196000000,5766000000,3239000000,...,-1076000000,-1417000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-12-31,NaN,NaN,NaN,NaN,NaN,NaN,27250000000,10795000000,4897000000,3040000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-31,6348000000,1445000000,930000000,1.31,1.29,0.525,28022000000,12106000000,5148000000,2848000000,...,-987000000,-275000000,-28687000000,-6038000000,-4003000000,-5.73,-5.70,-2.535,-9917000000,2196000000
2010-06-30,6731000000,1596000000,1121000000,1.57,1.54,0.525,28496000000,12891000000,5437000000,3022000000,...,-1438000000,-698000000,-28933000000,-6141000000,-3959000000,-5.66,-5.62,-2.550,-7700000000,2357000000
2010-09-30,6874000000,1576000000,1106000000,1.55,1.53,0.525,30065000000,14048000000,5395000000,4466000000,...,-940000000,-1162000000,-29328000000,-6279000000,-4138000000,-5.89,-5.82,-2.565,-4700000000,3217000000
2010-12-31,6709000000,1301000000,928000000,1.29,1.27,0.525,30156000000,12215000000,6089000000,3377000000,...,676000000,14000000,6709000000,1301000000,928000000,1.29,1.27,0.525,-1671000000,676000000
2011-03-31,7311000000,1578000000,1081000000,1.52,1.49,0.550,31129000000,12645000000,5900000000,2747000000,...,-757000000,-664000000,-32956000000,-7512000000,-5231000000,-7.32,-7.21,-2.600,-10204000000,4310000000
2011-06-30,7680000000,1655000000,1160000000,1.63,1.60,0.550,32282000000,13537000000,6203000000,3376000000,...,-809000000,-1437000000,-33167000000,-7417000000,-5112000000,-7.16,-7.05,-2.625,-7266000000,3577000000


In [40]:
window_size = 4
overlap = 1
num_windows = 4
[r[i:i + window_size].index for i in range(len(r) - window_size + 1 - num_windows,len(r) - window_size + 1, overlap)]

[DatetimeIndex(['2013-09-30', '2013-12-31', '2014-03-31', '2014-06-30'], dtype='datetime64[ns]', name=u'date', freq=None, tz=None),
 DatetimeIndex(['2013-12-31', '2014-03-31', '2014-06-30', '2014-09-30'], dtype='datetime64[ns]', name=u'date', freq=None, tz=None),
 DatetimeIndex(['2014-03-31', '2014-06-30', '2014-09-30', '2014-12-31'], dtype='datetime64[ns]', name=u'date', freq=None, tz=None),
 DatetimeIndex(['2014-06-30', '2014-09-30', '2014-12-31', '2015-03-31'], dtype='datetime64[ns]', name=u'date', freq=None, tz=None)]

In [2]:
# Add closing price and delta from next period i.e. did it go up 3 months from now
# instead make it more binary ie 1 is went up by X% using group df1 = df[df['budget'] > 10000]
# REMIND: Normalize annual number to quarterly?
XY = pandas.DataFrame()
for s in symbols['symbol']:
    try:
        r = reports[reports.symbol == s].sort(ascending=True)
        p = prices[prices.symbol == s].sort(ascending=True)
        r['close'] = r.index.map(lambda x: p.ix[p.index[p.index.searchsorted(x)]]['close'])
        r['delta'] = (r['close'].shift(-1) - r['close']).fillna(0)
        XY = XY.append(r)
    except:
        print "Problems parsing", s
print "Found", len(XY['symbol'].unique()), "symbols"

Problems parsing DISCK
Found 474 symbols


,symbol,amend,period_focus,fiscal_year,doc_type,revenues,op_income,net_income,eps_basic,eps_diluted,...,assets,cur_assets,cur_liab,cash,equity,cash_flow_op,cash_flow_inv,cash_flow_fin,close,delta
2009-06-30,MMM,False,Q2,2009,10-Q,5719000000,1191000000,783000000,1.12,1.12,...,26095000000,9898000000,5318000000,2596000000,11221000000,2161000000,-375000000,-1127000000,60.10,13.70
2009-09-30,MMM,False,Q3,2009,10-Q,6193000000,1478000000,957000000,1.36,1.35,...,27633000000,11196000000,5766000000,3239000000,12919000000,3897000000,-1076000000,-1417000000,73.80,8.87
2009-12-31,MMM,False,FY,2009,10-K,23123000000,4814000000,3193000000,4.56,4.52,...,27250000000,10795000000,4897000000,3040000000,13302000000,4941000000,-1732000000,-2014000000,82.67,0.90
2010-03-31,MMM,False,Q1,2010,10-Q,6348000000,1445000000,930000000,1.31,1.29,...,28022000000,12106000000,5148000000,2848000000,13851000000,1082000000,-987000000,-275000000,83.57,-4.58
2010-06-30,MMM,False,Q2,2010,10-Q,6731000000,1596000000,1121000000,1.57,1.54,...,28496000000,12891000000,5437000000,3022000000,14263000000,2220000000,-1438000000,-698000000,78.99,7.72


In [4]:
# Split into train/test sets
# Should use sklearn.cross_validation.StratifiedShuffleSplit to try and maintain industry sector % in each
# Or bin by financial size http://www.gregreda.com/2013/10/26/using-pandas-on-the-movielens-dataset/
# s = XY['symbol'].unique()
from sklearn.cross_validation import train_test_split
unique_symbols = pandas.unique(XY['symbol'].values.ravel())
train_symbols, test_symbols = train_test_split(unique_symbols, test_size = 0.2)
print "num train symbols:", len(train_symbols)
train = XY[XY['symbol'].isin(train_symbols)]
test = XY[XY['symbol'].isin(test_symbols)]
print "test/train", 1.0 * len(test)/len(train)

num train symbols: 379
test/train 0.253043685844


In [5]:
# Convert into pure numerical matrices for training by dropping non-numeric columsn and split into X/y
X_train = train.ix[:,5:-1].values
y_train = train['delta'].values
X_test = test.ix[:,5:-1].values
y_test = test['delta'].values

In [7]:
import numpy
with open("train_test.npz", "wb") as f:
    numpy.savez(f, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)